## Example ADVANCED mode recipe - normalization layer extra parameters tuning by SLaNC


In [1]:
import warnings
warnings.filterwarnings("ignore")


1. Instantiate a `torch` model from source, HF hub in this case.

In [2]:
from transformers import pipeline

model = "openai/clip-vit-base-patch32"
task = "zero-shot-image-classification"

task_cases = [
    dict(
        images="http://images.cocodataset.org/val2017/000000039769.jpg",
        candidate_labels=[
            "a photo of cats",
            "a photo of dogs",
        ],
    ),
    dict(
        images="http://images.cocodataset.org/val2017/000000397133.jpg",
        candidate_labels=[
            "a kitchen scene",
            "a living room scene",
        ],
    ),
]

pipe = pipeline(
    task=task,
    model=model,
    device_map="auto",
)

# -------------------------------------------------------------------------------
[pipe(**_tc) for _tc in task_cases]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


[[{'score': 0.9988459348678589, 'label': 'a photo of cats'},
  {'score': 0.0011540568666532636, 'label': 'a photo of dogs'}],
 [{'score': 0.9962789416313171, 'label': 'a kitchen scene'},
  {'score': 0.0037210225127637386, 'label': 'a living room scene'}]]

2. Transform into `DmxModel`; this does not change the functional behavior.

In [3]:
from dmx.compressor import DmxModel

pipe.model = DmxModel.from_torch(pipe.model)

# -------------------------------------------------------------------------------
[pipe(**_tc) for _tc in task_cases]

[[{'score': 0.9988459348678589, 'label': 'a photo of cats'},
  {'score': 0.0011540546547621489, 'label': 'a photo of dogs'}],
 [{'score': 0.9962789416313171, 'label': 'a kitchen scene'},
  {'score': 0.003721001325175166, 'label': 'a living room scene'}]]

3. Configure to BASIC mode; this should bring in all VSIMD approximations with default config.

In [4]:
pipe.model.to_basic_mode()

# -------------------------------------------------------------------------------
[pipe(**_tc) for _tc in task_cases]

[[{'score': 0.843508780002594, 'label': 'a photo of cats'},
  {'score': 0.1564912348985672, 'label': 'a photo of dogs'}],
 [{'score': 0.938322126865387, 'label': 'a kitchen scene'},
  {'score': 0.061677876859903336, 'label': 'a living room scene'}]]

4. SLaNC calibrate `LayerNorm` instances.

In [5]:
from dmx.compressor import nn
from dmx.compressor.advanced_recipe import (
    DmxSLaNCHyperparams,
    DmxSLaNCRecipe,
)


def hp_gen(_model) -> dict:
    _hp = {}    
    for _n, _m in _model.named_dmx_modules():
        if isinstance(_m, nn.LayerNorm):
            if "layer_norm1" in _n:
                _hp[_m] = DmxSLaNCHyperparams(
                    position="post_attn",
                    prev_layer=pipe.model.get_submodule(_n.replace("layer_norm1", "self_attn", -1)),
                )
            elif "layer_norm2" in _n:
                _hp[_m] = DmxSLaNCHyperparams(
                    position="post_mlp",
                    prev_layer=pipe.model.get_submodule(_n.replace("layer_norm2", "mlp", -1)),
                )
    return _hp

with DmxSLaNCRecipe(hp_gen).applied_to(pipe.model):
    print("SLaNC done!")

# -------------------------------------------------------------------------------
[pipe(**_tc) for _tc in task_cases]

SLaNC done!


[[{'score': 0.6110338568687439, 'label': 'a photo of cats'},
  {'score': 0.3889661133289337, 'label': 'a photo of dogs'}],
 [{'score': 0.5365557670593262, 'label': 'a living room scene'},
  {'score': 0.4634442627429962, 'label': 'a kitchen scene'}]]